In [1]:
import os
import numpy as np
import tensorflow as tf
from scipy.misc import imread

In [2]:
batch_size = 100

In [3]:
def mean_norm(dataset):
    """
    subtract mean and do normalization for each image
    Args:
        dataset: dataset with shape [N, H, W, C]
                 N data points, Height H, Width W, Channel C
    Return:
        dataset: processed dataset
    """
    N, H, W, C = dataset.shape
    X = dataset.reshape((N, -1))
    X -= np.mean(X, axis = 0)
    X /= np.std(X, axis = 0)
    X = X.reshape((N, H, W, C))
    return X

In [4]:
with open('tiny-imagenet-200/wnids.txt') as f:
    wnids = map(str.strip, list(f))

FileNotFoundError: [Errno 2] No such file or directory: 'tiny-imagenet-200/wnids.txt'

In [29]:
train = {
    'images': [],
    'labels': []
}

In [32]:
train['labels'] = [i for l in [[j] * 500 for j in range(200)] for i in l]

In [30]:
for wnid in wnids:
    subdir = 'tiny-imagenet-200/train/%s/images/' % wnid
    
    files = map(lambda s: subdir + s, os.listdir(subdir))
    train['images'].extend([imread(f, mode='RGB') for f in files])

In [33]:
len(train['labels'])

100000

In [36]:
img_train = mean_norm(np.array(train['images'], dtype=np.float32))
lab_train = np.array(train['labels'], dtype=np.int32)

In [38]:
img_train[0].shape

(64, 64, 3)

In [70]:
x = tf.placeholder(tf.float32, [None, 64, 64, 3])
W = tf.Variable(tf.zeros([3, 3, 3, 64]))
b = tf.Variable(tf.zeros([64]))
conv_out = tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
h_conv1 = tf.nn.relu(conv_out + b)
# y = tf.matmul(x, W) + b
y_conv=tf.nn.softmax(tf.matmul(h_conv1, W) + b)
y_ = tf.placeholder(tf.float32, [None, 64])
# train_step = tf.train.AdamOptimizer(learning_rate=0.5).minimize(loss)

ValueError: Shape (?, 64, 64, 64) must have rank 2

In [52]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [55]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())

In [56]:
for i in xrange(0, len(img_train), batch_size):
    train_step.run(feed_dict={x: img_train, y_: lab_train})

ValueError: Cannot feed value of shape (100000, 64, 64, 3) for Tensor u'Placeholder_4:0', which has shape '(?, 4096)'

In [59]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [62]:
batch = mnist.train.next_batch(100)
batch[0].shape, batch[1].shape[1]
img_train.shape, lab_train.shape

((100000, 64, 64, 3), (100000,))